In [120]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.stattools import pacf, acf
from statsmodels.tsa.seasonal import seasonal_decompose

In [121]:
# read raw tourism file
data_dir = "C:/Users/benlc/OneDrive/Desktop/python_learn/tourism-demand-malaysia/data/raw/"
tourism_raw = pd.read_csv(data_dir+"Tourist Arrivals _ Tourism Malaysia [2021-Aug-31 03.22 AM].csv",nrows=21)
tourism_raw.tail()

,Year,Destination,Jan - No. of Arrivals,Feb - No. of Arrivals,Mar - No. of Arrivals,Apr - No. of Arrivals,May - No. of Arrivals,Jun - No. of Arrivals,Jul - No. of Arrivals,Aug - No. of Arrivals,...,Mar - Percentage Change Year on Year,Apr - Percentage Change Year on Year,May - Percentage Change Year on Year,Jun - Percentage Change Year on Year,Jul - Percentage Change Year on Year,Aug - Percentage Change Year on Year,Sep - Percentage Change Year on Year,Oct - Percentage Change Year on Year,Nov - Percentage Change Year on Year,Dec - Percentage Change Year on Year
16,2016,Malaysia,2376166,2091098,2198716,2101280,2144119,2121396,2296615,2282173,...,-1.933966,1.416945,1.190671,12.018427,3.635569,4.565194,1.632556,11.696432,2.467758,2.451860
17,2017,Malaysia,2350270,2043215,2238184,2145734,2039016,2134647,2263478,2129013,...,1.795048,2.115568,-4.901920,0.624636,-1.442863,-6.711148,-1.226841,-11.067846,-2.249089,-7.981155
18,2018,Malaysia,2276750,2050613,2192855,1957248,1976981,2275921,2305324,2253534,...,-2.025258,-8.784220,-3.042399,6.618143,1.848748,5.848767,0.215592,1.690676,-0.902556,-3.413665
19,2019,Malaysia,2195684,2165933,2334613,2159517,2098267,2400561,2415097,2342438,...,6.464541,10.334357,6.134910,5.476464,4.761717,3.945092,-4.759241,-3.459024,-1.031589,-15.360934
20,2020,Malaysia,2164459,1397912,671084,7546,5411,6585,18660,11631,...,-71.255022,-99.650570,-99.742121,-99.725689,-99.227360,-99.503466,-99.192276,-99.442940,-99.420103,-99.469230


In [122]:
# clean raw data and transform to long format
list_column = [0] + list(range(2,14))
column_name = ["year", "1","2","3","4","5","6","7","8","9","10","11","12"]

# select and rename column
tourism_clean = tourism_raw.iloc[:,list_column]
tourism_clean.columns = column_name

# change to longer format
tourism_clean = tourism_clean.melt(id_vars='year', var_name = "month", value_name = "tourist_arrival")

# convert to date
tourism_clean['day'] = 1
tourism_clean['date'] = pd.to_datetime(tourism_clean[['year', 'month','day']])
tourism_clean = tourism_clean[['date','tourist_arrival']].sort_values(by="date").reset_index(drop=True)
tourism_clean.tail()

,date,tourist_arrival
247,2020-08-01,11631
248,2020-09-01,16131
249,2020-10-01,11315
250,2020-11-01,11420
251,2020-12-01,10568


In [123]:
## export to intermediate clean dataset
# tourism_clean.to_csv("C:/Users/benlc/OneDrive/Desktop/python_learn/tourism-demand-malaysia/data/interim/tourism_clean.csv")


In [124]:
tourism_clean.set_index("date", inplace=True)
tourism_clean.head()

,tourist_arrival
date,
2000-01-01,731509
2000-02-01,786040
2000-03-01,737678
2000-04-01,916382
2000-05-01,894350


Explorenatary data analysis
- Try to understand existing data and identify which transformation or features are useful for 12 months ahead forecasting

List of EDA tasks
1) target variable viz & stationary check & distribution
2) acf and pacf, decompositon & seasonality check - multiple correlation 
3) external variable cross correlation

In [125]:
# tourism_clean.head()
# tourism_clean.reset_index()


In [126]:
## check stationary and visualise plot and histogram
def check_stationary(timeseries):
    dftest = ts.adfuller(timeseries)
    dfoutput = pd.Series(dftest[0:4], 
                            index=['Test Statistic','p-value','Lags Used','Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

check_stationary(timeseries = tourism_clean['tourist_arrival'])

def chunk_stats_fn(ori_df, y_col, n_chunks = 10):

    # split into even chunks
    def split(a, n):
        k, m = divmod(len(a), n)
        return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

    chunks = list(split(ori_df[y_col].to_numpy(), n_chunks))

    # Calculate the stats: mean and variance
    data = []
    for i, chunk in enumerate(chunks, 1):
        chunk_ind= i
        chunk_mean = np.mean(chunk)
        chunk_var = np.var(chunk)
        data.append([chunk_ind, chunk_mean, chunk_var])

    split_chunks_stats = pd.DataFrame(data, columns=['Chunk','Mean', 'Variance'])
    
    return split_chunks_stats

output = chunk_stats_fn(ori_df = tourism_clean, y_col = "tourist_arrival", n_chunks = 5)
print(output)

def display_time_series(ori_df, y_col):
    df = ori_df.copy()

    # Determing rolling statistics
    df[y_col+'_rolmean'] = df[y_col].rolling(window=12).mean()
    df[y_col+'_rolstd'] = df[y_col].rolling(window=12).std()
    df = df.reset_index()
    df = df.melt( id_vars='date', var_name = "type", value_name = "value")

    fig = px.line(df, x="date", y="value", color="type")
    fig.show()

display_time_series(ori_df = tourism_clean, y_col = "tourist_arrival")

def display_histogram(df, x_col):
    fig = px.histogram(df, x=x_col)
    fig.show()

display_histogram(df = tourism_clean, x_col = "tourist_arrival")






Test Statistic           -1.116766
p-value                   0.708340
Lags Used                12.000000
Observations Used       239.000000
Critical Value (1%)      -3.458011
Critical Value (5%)      -2.873710
Critical Value (10%)     -2.573256
dtype: float64
   Chunk          Mean      Variance
0      1  9.991981e+05  5.181011e+10
1      2  1.522328e+06  4.245026e+10
2      3  2.004138e+06  2.969928e+10
3      4  2.195666e+06  3.047893e+10
4      5  1.738306e+06  7.285986e+11


In [127]:
#line + (table + histogram)
def display_time_series_obj(df, y_col):
    return(go.Scatter(x=df.index, y=df[y_col],
                    mode='lines',
                    name='markers'))

def display_histogram_obj(df, y_col):
    return(go.Histogram(x=df[y_col]))

# Initialize figure with subplots
fig = make_subplots(rows=1, cols=2, column_widths=[0.7, 0.3])

fig.add_trace(
    display_time_series_obj(df=tourism_clean, y_col = "tourist_arrival"),
    row=1, col=1
)
fig.add_trace(
    display_histogram_obj(df=tourism_clean, y_col ='tourist_arrival'),
    row=1, col=2
)


fig.show()

In [128]:
def create_corr_plot(series, plot_pacf=False):
    corr_array = pacf(series.dropna(), alpha=0.05) if plot_pacf else acf(series.dropna(), alpha=0.05)
    lower_y = corr_array[1][:,0] - corr_array[0]
    upper_y = corr_array[1][:,1] - corr_array[0]

    fig = go.Figure()
    [fig.add_scatter(x=(x,x), y=(0,corr_array[0][x]), mode='lines',line_color='#3f3f3f') 
     for x in range(len(corr_array[0]))]
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=corr_array[0], mode='markers', marker_color='#1f77b4',
                   marker_size=12)
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=upper_y, mode='lines', line_color='rgba(255,255,255,0)')
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=lower_y, mode='lines',fillcolor='rgba(32, 146, 230,0.3)',
            fill='tonexty', line_color='rgba(255,255,255,0)')
    fig.update_traces(showlegend=False)
    fig.update_xaxes(range=[-1,42])
    fig.update_yaxes(zerolinecolor='#000000')
    
    title='Partial Autocorrelation (PACF)' if plot_pacf else 'Autocorrelation (ACF)'
    fig.update_layout(title=title)

    fig.show()

create_corr_plot(tourism_clean['tourist_arrival'])
create_corr_plot(tourism_clean['tourist_arrival'], plot_pacf=True)




C:\Users\benlc\anaconda3\envs\data-science\lib\site-packages\statsmodels\tsa\stattools.py:657: FutureWarning:

The default number of lags is changing from 40 tomin(int(10 * np.log10(nobs)), nobs - 1) after 0.12is released. Set the number of lags to an integer to  silence this warning.

C:\Users\benlc\anaconda3\envs\data-science\lib\site-packages\statsmodels\tsa\stattools.py:667: FutureWarning:

fft=True will become the default after the release of the 0.12 release of statsmodels. To suppress this warning, explicitly set fft=False.



C:\Users\benlc\anaconda3\envs\data-science\lib\site-packages\statsmodels\tsa\stattools.py:1024: FutureWarning:

The default number of lags is changing from 40 tomin(int(10 * np.log10(nobs)), nobs // 2 - 1) after 0.12is released. Set the number of lags to an integer to  silence this warning.



In [129]:
def create_corr_plot(series, plot_pacf=False):
    corr_array = pacf(series.dropna(), alpha=0.05) if plot_pacf else acf(series.dropna(), alpha=0.05)
    lower_y = corr_array[1][:,0] - corr_array[0]
    upper_y = corr_array[1][:,1] - corr_array[0]

    fig = go.Figure()
    [fig.add_scatter(x=(x,x), y=(0,corr_array[0][x]), mode='lines',line_color='#3f3f3f') 
     for x in range(len(corr_array[0]))]
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=corr_array[0], mode='markers', marker_color='#1f77b4',
                   marker_size=12)
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=upper_y, mode='lines', line_color='rgba(255,255,255,0)')
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=lower_y, mode='lines',fillcolor='rgba(32, 146, 230,0.3)',
            fill='tonexty', line_color='rgba(255,255,255,0)')
    fig.update_traces(showlegend=False)
    fig.update_xaxes(range=[-1,42])
    fig.update_yaxes(zerolinecolor='#000000')
    
    title='Partial Autocorrelation (PACF)' if plot_pacf else 'Autocorrelation (ACF)'
    fig.update_layout(title=title)
    
    return fig


acf_plot_obj = create_corr_plot(tourism_clean['tourist_arrival'])
pacf_plot_obj = create_corr_plot(tourism_clean['tourist_arrival'], plot_pacf=True)

fig = make_subplots(rows=1, cols=2,
    subplot_titles=('Autocorrelation (ACF)', 'Partial Autocorrelation (PACF)'))

for t in acf_plot_obj.data:
    fig.append_trace(t, row=1, col=1)

for t in pacf_plot_obj.data:
    fig.append_trace(t, row=1, col=2)


# fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")
fig.show()

In [130]:
tourism_monthly = tourism_clean.resample("M").mean()
tourism_monthly["month"] = tourism_monthly.index.month
tourism_quarterly = tourism_clean.resample("Q").mean()
tourism_quarterly["quarter"] = tourism_quarterly.index.quarter

fig = px.violin(tourism_monthly, y="tourist_arrival", x="month", box=True, points="all")
fig.show()

fig = px.violin(tourism_quarterly, y="tourist_arrival", x="quarter", box=True, points="all")
fig.show()

In [131]:
ss_decomposition_add = seasonal_decompose(x=tourism_clean, 
                                          model='additive', 
                                          period=12)
estimated_obs = ss_decomposition_add.observed
estimated_trend_add = ss_decomposition_add.trend
estimated_seasonal_add = ss_decomposition_add.seasonal
estimated_residual_add = ss_decomposition_add.resid

fig = px.line(estimated_obs)
fig.show()
fig = px.line(estimated_trend_add)
fig.show()
fig = px.line(estimated_seasonal_add)
fig.show()
fig = px.line(estimated_residual_add)
fig.show()

# fig = go.Figure()
# fig.add_scatter(x=estimated_obs.index, y = estimated_obs.values,
#         mode='lines', name='markers')
# fig.add_scatter(x=estimated_trend_add.index, y = estimated_trend_add.values,
#         mode='lines', name='markers')
# fig.add_scatter(x=estimated_seasonal_add.index, y = estimated_seasonal_add.values,
#         mode='lines', name='markers')
# fig.add_scatter(x=estimated_residual_add.index, y = estimated_residual_add.values,
#         mode='lines', name='markers')
# fig.show()



In [132]:
from pandas_datareader import data
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau
# get USD MYR currency
myr_usd = data.DataReader('DEXMAUS', 'fred',  start="2010-01-01" )
myr_usd.index.name = "date"




In [176]:
# convert to monthly average 
myr_usd_month = myr_usd.resample('MS').mean()

# merge tourism demand with MYR USD conversion
tourism_myr = tourism_clean.merge(myr_usd_month, how="left", on='date').dropna()
tourism_myr['year'] = tourism_myr.index.year
tourism_myr['year'] = tourism_myr['year'].astype(str)
tourism_myr.tail()

,tourist_arrival,DEXMAUS,year
date,,,
2020-08-01,11631,4.184129,2020
2020-09-01,16131,4.147838,2020
2020-10-01,11315,4.150024,2020
2020-11-01,11420,4.113161,2020
2020-12-01,10568,4.054567,2020


In [177]:
scaler = StandardScaler()
for col in ['tourist_arrival','DEXMAUS']:
    tourism_myr[col] = scaler.fit_transform(tourism_myr[col].values.reshape(-1,1))


tourism_myr.head()



,tourist_arrival,DEXMAUS,year
date,,,
2010-01-01,-0.154061,-0.604714,2010
2010-02-01,-0.265643,-0.528052,2010
2010-03-01,0.062948,-0.712076,2010
2010-04-01,-0.186842,-0.940808,2010
2010-05-01,0.010604,-0.849182,2010


In [184]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

def lag_plot(ori_df, x, y, lags_list, z=None):

    lags = lags_list
    for lag in lags:
        df = ori_df.copy()

        col_name = x+"_lag_"+str(lag)
        df[col_name] = df[x].shift(lag)
        df.dropna(inplace=True)

        pearson_score, p_value = pearsonr(df[col_name], df[y])
        title_name = f'Pearson correlation: {pearson_score:.2f}, p-value : {p_value:.2f}'

        df_ts = df.reset_index().drop([z,x],axis = 1)
        scaler = StandardScaler()
        minmax_scaler = MinMaxScaler()
        for col in [col_name,y]:
            df_ts[col] = minmax_scaler.fit_transform(df_ts[col].values.reshape(-1,1))

        df_ts = df_ts.melt(id_vars='date', var_name = "type", value_name = "value")

        print(df.head())
        fig = px.scatter(df, x=col_name, y=y, color=z, title=title_name,  width=800, height=400)
        fig.show()

        fig = px.line(df_ts, x='date', y='value', color='type',   width=800, height=400)
        fig.show()


lag_plot(ori_df = tourism_myr, x = "DEXMAUS", y= "tourist_arrival" , z="year", lags_list= [3,6,9])



            tourist_arrival   DEXMAUS  year  DEXMAUS_lag_3
date                                                      
2010-04-01        -0.186842 -0.940808  2010      -0.604714
2010-05-01         0.010604 -0.849182  2010      -0.528052
2010-06-01         0.448876 -0.824847  2010      -0.712076
2010-07-01         0.393632 -0.942550  2010      -0.940808
2010-08-01         0.195730 -1.045015  2010      -0.849182


            tourist_arrival   DEXMAUS  year  DEXMAUS_lag_6
date                                                      
2010-07-01         0.393632 -0.942550  2010      -0.604714
2010-08-01         0.195730 -1.045015  2010      -0.528052
2010-09-01         0.116161 -1.140300  2010      -0.712076
2010-10-01         0.261780 -1.148734  2010      -0.940808
2010-11-01         0.164421 -1.120700  2010      -0.849182


            tourist_arrival   DEXMAUS  year  DEXMAUS_lag_9
date                                                      
2010-10-01         0.261780 -1.148734  2010      -0.604714
2010-11-01         0.164421 -1.120700  2010      -0.528052
2010-12-01         0.236372 -1.090825  2010      -0.712076
2011-01-01        -0.116360 -1.226499  2011      -0.940808
2011-02-01        -0.547130 -1.254384  2011      -0.849182


1) Feature engineering - month, quarter, number of holidays, big festive days
2) create modeling pipeline
    - resample
    - scaler, preprocessing pipeline
    - modelling
    - 
3) Modeling with arima, arimax, ets, ML 
    - hyperparameter tuning
    - model selection
    - ensembling
    - evaluation